In [5]:
import glob, json
import concurrent.futures as cf
import re
import ast
import dask
import time
import MeCab as mcb
import dask.bag as db
import concurrent.futures as cf
import dask.multiprocessing
import os
import pyspark

sc = pyspark.SparkContext(appName='tweeter')

#mecab = mcb.Tagger('-Ochasen')
#mecab.parse('')
    
with open ('emoji-test.txt', 'r') as f:
    emoji = f.readlines()
    emoji = [x[x.rfind('#')+2:x.rfind('#')+3] for x in emoji if x != '' and x[0] != '#' and x != '\n']

In [2]:
def tweet_extractor(line):
    line = line.rstrip()
    date = line[:10].replace('_', '-')
    jsons = ast.literal_eval(line[14:])
    json_lst = [date + list(x.values())[0].replace('\n', '。') for x in jsons]
#     for j in json_lst:
#         cleaner(j)
    return json_lst

In [7]:
def cleaner(json):
    mecab = mcb.Tagger('-Ochasen')
    mecab.parse('')
    # remove emoji
    date = json[:10]
    line = json[10:]
    line = ''.join([x for x in line if x not in emoji])

    # remove links
    links = re.findall(r'.*?(https://t.co/[a-zA-Z0-9]*).*?', line)
    for l in links:
        line = line.replace(l, '')
    
    # there could be multiple tweets so split them first
    line = line.split('",')
    
    # remove user id
    users = [re.findall(r'.*?([@＠].+) ?.*?', x) for x in line]
    users = sum(users, [])
    if len(users) > 0:
        for l in line:
            for u in users:
                l = l.replace('u', '')
    line = '。'.join(line)
    
    # hashtag
    tags = re.findall(r'[#＃]{1}[\S^#^＃]+', line)
    if len(tags) > 0:
        for t in tags:
            line = line.replace(t, ' ')
        tags = list(set(tags))
    tag_lst = ' '.join(tags)
    
    # MeCab
    nodes = mecab.parseToNode(line)
    tokenized = []
    while nodes:
        features = nodes.feature.split(',')
        if features[0]!='記号':
            tokenized.append(features[6])
        nodes = nodes.next
    token_filtered = [x for x in tokenized if x != '*' and x != '。']
    
    # summary: for now don't use hashtags
    result = ' '.join(token_filtered) #+ ' ' + tag_lst
    
    if len(token_filtered) + len(tags) > 1:
        tweet_processed = result.replace('。', ' ').replace('*', ' ')
        #return tweet_processed
        with open ('./all_2.csv', 'a') as f:
            f.write(tweet_processed + '\n')
    else:
        pass

In [8]:
import dask.multiprocessing
import os
import pyspark

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

def delay(json):
    return dask.delayed(cleaner)(json)

#sc = pyspark.SparkContext(appName='tweeter')
files = sc.textFile('./test/part-*')

start = time.time()
print ('Starting at: ', time.strftime("%H:%M:%S", time.gmtime(start)))

graphy = files.map(tweet_extractor).flatMap(lambda x: x).map(cleaner)
graphy.collect()

# for c in chunks:
#     bags = dask.bag.read_text(c, blocksize=50000000)
#     test = bags.map(tweet_extractor)#.flatten().map(cleaner)
    
#     test.compute()

# timing the computation
elapsed_time = time.time() - start
to_print = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print ('Time consumed on computing: '.format(c), to_print)

Starting at:  07:40:08


KeyboardInterrupt: 